In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI

import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess


In [2]:
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise RuntimeError("Missing ANTHROPIC_API_KEY in .env")
from anthropic import Anthropic
client = Anthropic(api_key=api_key)
MODEL = "claude-sonnet-4-20250514"

In [3]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [4]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt



In [5]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [6]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)


In [11]:
def optimize_claude(python):
    result = client.messages.stream(
        model=MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)


In [12]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""


In [9]:


exec(pi)



Result: 3.141592658589
Execution Time: 12.747629 seconds


In [13]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <chrono>
#include <iomanip>

double calculate(long long iterations, long long param1, long long param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        long long j1 = i * param1 - param2;
        long long j2 = i * param1 + param2;
        result -= 1.0 / j1;
        result += 1.0 / j2;
    }
    return result;
}

int main() {
    auto start = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4, 1) * 4;
    auto end = std::chrono::high_resolution_clock::now();
    
    auto duration = std::chrono::duration<double>(end - start);
    
    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << duration.count() << " seconds" << std::endl;
    
    return 0;
}
```

In [14]:
exec(pi)


Result: 3.141592658589
Execution Time: 13.242483 seconds


In [15]:
!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

/bin/bash: line 1: clang++: command not found
/bin/bash: line 1: ./optimized: No such file or directory


In [16]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [17]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 48.723485 seconds


In [18]:
optimize_claude(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <algorithm>
#include <chrono>
#include <iomanip>

class LCG {
private:
    uint64_t value;
    static constexpr uint64_t a = 1664525ULL;
    static constexpr uint64_t c = 1013904223ULL;
    static constexpr uint64_t m = 4294967296ULL; // 2^32

public:
    LCG(uint64_t seed) : value(seed) {}
    
    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

long long max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    std::vector<int> random_numbers;
    random_numbers.reserve(n);
    
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        random_numbers.push_back(static_cast<int>(lcg_gen.next() % range) + min_val);
    }
    
    long long max_sum = LLONG_MIN;
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            max_sum = std::

In [47]:
with gr.Blocks() as ui :
    with gr.Row():
        title = gr.Markdown("<h1 style='font-family:TimesNewRoman; font-size:32px; color:Green; text-align:center;'> CLAUDE IS HERE </h1>")
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)

    with gr.Row():
        
        convert = gr.Button("Convert Now")

    convert.click(optimize_claude, inputs=[python], outputs=[cpp])
        
do = ui.launch()    

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <iomanip>
#include <limits>

class LCG {
private:
    uint64_t value;
    static constexpr uint64_t a = 1664525;
    static constexpr uint64_t c = 1013904223;
    static constexpr uint64_t m = 1ULL << 32;

public:
    LCG(uint64_t seed) : value(seed) {}
    
    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

long long max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    int range = max_val - min_val + 1;
    
    long long max_sum = std::numeric_limits<long long>::min();
    long long current_sum = 0;
    
    for (int i = 0; i < n; i++) {
        int val = static_cast<int>(lcg.next() % range) + min_val;
        current_sum = std::max(static_cast<long long>(val), current_sum + val);
        max_sum = std::max(max_sum, current_sum);
    }
    
    return max_sum;
}

long long total_max_subarray_sum(int n, uint64_t initial_seed, int min_val,